In [27]:
import numpy as np

In [28]:
import pandas as pd
from pandas import DataFrame,Series

In [29]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()


,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [30]:
df['labels'].unique()

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [31]:
def newsType(val):
    if val == 'business':
        return 1
    elif val == 'entertainment':
        return 2
    elif val == 'politics':
        return 3
    elif val == 'sports':
        return 4
    else:
        return 5


In [32]:
df['NewsLabel'] = df['labels'].apply(newsType)
df.head()

,text,labels,NewsLabel
0,Ad sales boost Time Warner profit\n\nQuarterly...,business,1
1,Dollar gains on Greenspan speech\n\nThe dollar...,business,1
2,Yukos unit buyer faces loan claim\n\nThe owner...,business,1
3,High fuel prices hit BA's profits\n\nBritish A...,business,1
4,Pernod takeover talk lifts Domecq\n\nShares in...,business,1


In [33]:
df.drop(columns=['labels'], axis=1, inplace=True)

df.head()

,text,NewsLabel
0,Ad sales boost Time Warner profit\n\nQuarterly...,1
1,Dollar gains on Greenspan speech\n\nThe dollar...,1
2,Yukos unit buyer faces loan claim\n\nThe owner...,1
3,High fuel prices hit BA's profits\n\nBritish A...,1
4,Pernod takeover talk lifts Domecq\n\nShares in...,1


In [34]:
df.drop_duplicates(inplace = True)
import string
from nltk.corpus import stopwords

def preprocess(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]


In [35]:
df['text'].head().apply(preprocess)

,text
0,"[Ad, sales, boost, Time, Warner, profit, Quart..."
1,"[Dollar, gains, Greenspan, speech, dollar, hit..."
2,"[Yukos, unit, buyer, faces, loan, claim, owner..."
3,"[High, fuel, prices, hit, BAs, profits, Britis..."
4,"[Pernod, takeover, talk, lifts, Domecq, Shares..."


In [36]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer = preprocess)

In [37]:
count.fit(df['text'])

CountVectorizer(analyzer=<function preprocess at 0x7948c7ef70a0>)

In [42]:
textCount = count.transform(df['text'])

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer

In [48]:
tfidf_transformer = TfidfTransformer().fit(textCount)

In [49]:
message_tfidf = tfidf_transformer.transform(textCount)

In [52]:
print (message_tfidf)

  (0, 153)	0.06959734635877173
  (0, 187)	0.06959734635877173
  (0, 199)	0.06354433864487749
  (0, 803)	0.03384289201842407
  (0, 806)	0.03327930007540634
  (0, 823)	0.05191693904731793
  (0, 831)	0.02650634113081786
  (0, 1087)	0.07866461557860113
  (0, 1128)	0.06959734635877173
  (0, 1189)	0.055542697689310784
  (0, 1287)	0.06959734635877173
  (0, 1475)	0.06959734635877173
  (0, 1557)	0.06959734635877173
  (0, 1636)	0.03452370242224341
  (0, 1646)	0.05325156391395733
  (0, 1868)	0.06959734635877173
  (0, 1872)	0.04280513165482177
  (0, 2023)	0.04322105046112319
  (0, 2085)	0.08332894414991744
  (0, 2356)	0.2630220267995905
  (0, 2357)	0.13919469271754345
  (0, 2460)	0.0660565638295677
  (0, 2666)	0.04789754068788501
  (0, 3766)	0.06959734635877173
  (0, 4257)	0.05749133093098326
  :	:
  (2126, 37729)	0.01910647723995558
  (2126, 37753)	0.013305417676363914
  (2126, 37758)	0.015573671648680659
  (2126, 37779)	0.01390414236329901
  (2126, 37781)	0.05623404290414653
  (2126, 37790)	0.00

In [64]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(message_tfidf,df['NewsLabel'])

In [66]:
X=message_tfidf
Y=df['NewsLabel']

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [67]:
model.fit(X_train,Y_train)

MultinomialNB()

In [68]:
predict = model.predict(X_test)

In [69]:
expected = Y_test

In [70]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predict,expected))


0.8341158059467919
